# Explore LAT Data (for Burst)

In this example, we will examine the LAT data for a gamma-ray burst based on the time and position derived from a GBM trigger.

# Prerequisites

It is assumed that:

* You are in your working directory.
* The GBM reported a burst via a [GCN circular]() with the following information:
    * Name = GRB 080916C
    * RA = 121.8
    * Dec = -61.3
    * TStart = 243216766 s (Mission Elapsed Time)
    * T90 = 66 s

# Steps

The analysis steps are:

1. Extract the Data
2. Data Selections
3. Bin the Data
4. Look at the Data

## 1. Extract the Data

Generally, one should refer to [Extract LAT data](https://fermi.gsfc.nasa.gov/ssc/data/p6v11/analysis/scitools/extract_latdata.html) tutorial and use the time and spatial information from some source (here, a GCN notice) to make the appropiate extraction cuts from the [LAT data server](http://fermi.gsfc.nasa.gov/cgi-bin/ssc/LAT/LATDataQuery.cgi).

We use the following parameters to extract the data for an ROI of 20 degrees from 500s before to 1500 seconds after the trigger time:

* Search Center (RA,Dec)	=	(121.8,-61.3)
* Radius	=	20 degrees
* Start Time (MET)	=	243216266 seconds (2008-09-16T00:04:26)
* Stop Time (MET)	=	243218266 seconds (2008-09-16T00:37:46)
* Minimum Energy	=	20 MeV
* Maximum Energy	=	300000 MeV

Note that for analyses that require the diffuse background, the standard model starts at 60 MeV.

In [1]:
!wget https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/exploreLATDataGRB/LAT_explore_GRB.tgz

--2025-09-04 10:17:56--  https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/exploreLATDataGRB/LAT_explore_GRB.tgz
Resolving fermi.gsfc.nasa.gov (fermi.gsfc.nasa.gov)... 129.164.179.26
Connecting to fermi.gsfc.nasa.gov (fermi.gsfc.nasa.gov)|129.164.179.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 100685 (98K) [application/x-tar]
Saving to: ‘LAT_explore_GRB.tgz’

LAT_explore_GRB.tgz 100%[===================>]  98.33K  --.-KB/s    in 0.008s  

2025-09-04 10:17:57 (11.6 MB/s) - ‘LAT_explore_GRB.tgz’ saved [100685/100685]



In [2]:
!tar xvzf LAT_explore_GRB.tgz

x GRB081916C_counts_map.fits
x GRB081916C_lc_events.fits
x GRB081916C_light_curve.fits
x GRB081916C_map_events.fits
x grb_events.fits
x grb_spacecraft.fits


In [3]:
!mkdir data
!mv *.fits ./data

In [4]:
!ls ./data

grb_events.fits             GRB081916C_lc_events.fits
grb_spacecraft.fits         GRB081916C_light_curve.fits
GRB081916C_counts_map.fits  GRB081916C_map_events.fits


This will extract the data files used in this tutorial into the `data` directory.

## Data Selections

NOTE: For information on the recommended selections for burst analysis of LAT data, you should refer to the [Cicerone](http://fermi.gsfc.nasa.gov/ssc/data/p6v11/analysis/documentation/Cicerone/Cicerone_Data_Exploration/Data_preparation.html).

To map the region of the burst, you should select a large spatial region from within a short time range bracketing the burst. For the lightcurve on the other hand, its best to select you want a small region around the burst from within a long time range. Therefore, you will need to make two different data selections.

In both cases, we will use the loosest event class cut (to include all class events). This works because rapid, bright events like GRBs overwhelm the background rates for the short period of the flare.

Here, we use [gtselect](https://fermi.gsfc.nasa.gov/ssc/data/p6v11/analysis/scitools/help/gtselect.txt) to extract the region for the spatial mapping:

In [5]:
%%bash
gtselect evclass=128
    ./data/grb_events.fits
    ./data/GRB081916C_map_events.fits
    121.8
    -61.3
    20
    243216666
    243216966
    100
    300000
    180

Input FT1 file[@./data/events.txt]     ./data/grb_events.fits
Output FT1 file[./data/3C279_region_filtered.fits]     ./data/GRB081916C_map_events.fits
RA for new search center (degrees) (0:360) [193.98]     121.8
Dec for new search center (degrees) (-90:90) [-5.82]     -61.3
radius of new search region (degrees) (0:180) [20]     20
start time (MET in s) (0:) [INDEF]     243216666
end time (MET in s) (0:) [INDEF]     243216966
lower energy limit (MeV) (0:) [100]     100
upper energy limit (MeV) (0:) [500000]     300000
maximum zenith angle value (degrees) (0:180) [90]     180
Done.


**Notes**:
* The input photon file is `grb_events.fits`, and the output file is `GRB081916C_map_events.fits`.
* We selected a circular region with radius 20 degree around the burst location (the ROI here has to fall within that selected in the data server) from a 300s time period around the trigger.
* We made an additional energy cut, selecting only photons between 100 MeV and 300 GeV. This removes the low-energy high-background events from the map.

**Run [gtselect](https://fermi.gsfc.nasa.gov/ssc/data/p6v11/analysis/scitools/help/gtselect.txt) again**, this time with a smaller region but a longer time range. (Note that _gtselect_ saves the values from the previous run and uses them as defaults for the next run.)

In [6]:
%%bash
gtselect evclass=128
    ./data/grb_events.fits
    ./data/GRB081916C_lc_events.fits
    121.8
    -61.3
    10
    243216266
    243218266
    30
    300000
    180

Input FT1 file[./data/grb_events.fits]     ./data/grb_events.fits
Output FT1 file[./data/GRB081916C_map_events.fits]     ./data/GRB081916C_lc_events.fits
RA for new search center (degrees) (0:360) [121.8]     121.8
Dec for new search center (degrees) (-90:90) [-61.3]     -61.3
radius of new search region (degrees) (0:180) [20]     10
start time (MET in s) (0:) [243216666]     243216266
end time (MET in s) (0:) [243216966]     243218266
lower energy limit (MeV) (0:) [100]     30
upper energy limit (MeV) (0:) [300000]     300000
maximum zenith angle value (degrees) (0:180) [180]     180
Done.


**Notes**:

* A new output file, with the extension `lc_events.fits` has been produced.
* The search radius was reduced to 10 degrees.
* The start to stop time range has been expanded.
* The energy range has been expanded.

## Bin the Data

Use [gtbin](https://fermi.gsfc.nasa.gov/ssc/data/p6v11/analysis/scitools/help/gtbin.txt) to bin the photon data into a map and a lightcurve.

First, create the counts map:

In [7]:
%%bash
gtbin
    CMAP
    ./data/GRB081916C_map_events.fits
    ./data/GRB081916C_counts_map.fits
    NONE
    50
    50
    0.5
    CEL
    121.8
    -61.3
    0.
    AIT

This is gtbin version HEAD
Type of output file (CCUBE|CMAP|LC|PHA1|PHA2|HEALPIX) [CMAP]     CMAP
Event data file name[./data/3C279_region_filtered_gti.fits]     ./data/GRB081916C_map_events.fits
Output file name[./data/3C279_region_cmap.fits]     ./data/GRB081916C_counts_map.fits
Spacecraft data file name[NONE]     NONE
Size of the X axis in pixels[400]     50
Size of the Y axis in pixels[400]     50
Image scale (in degrees/pixel)[0.1]     0.5
Coordinate system (CEL - celestial, GAL -galactic) (CEL|GAL) [CEL]     CEL
8
Second coordinate of image center in degrees (DEC or galactic b)[-5.82]     -61.3
Rotation angle of image axis, in degrees[0]     0.
Projection method e.g. AIT|ARC|CAR|GLS|MER|NCP|SIN|STG|TAN:[AIT]     AIT


gtbin: WARNING: No spacecraft file: EXPOSURE keyword will be set equal to ontime.


**Notes**:
    
* Select _cmap_ (i.e., count map) as the output.
* When we ran _gtselect_ we called the file from a large area `GRB081916C_map_events.fits`.
* The counts map file will be called `GRB081916C_counts_map.fits`.
* Although in _gtselect_ we selected a circular region with a 20 degree radius, we will form a counts map with 50 pixels on a side, and 1/2 degree square pixels.

Now, we create the lightcurve. Once again, the previous inputs were saved and provided as defaults where appropriate.

In [8]:
%%bash
gtbin
    LC
    ./data/GRB081916C_lc_events.fits
    ./data/GRB081916C_light_curve.fits
    NONE
    LIN
    243216266
    243218266
    10

This is gtbin version HEAD
Type of output file (CCUBE|CMAP|LC|PHA1|PHA2|HEALPIX) [CMAP]     LC
Event data file name[./data/GRB081916C_map_events.fits]     ./data/GRB081916C_lc_events.fits
Output file name[./data/GRB081916C_counts_map.fits]     ./data/GRB081916C_light_curve.fits
Spacecraft data file name[NONE]     NONE
Algorithm for defining time bins (FILE|LIN|SNR) [LIN]     LIN
Start value for first time bin in MET[243215766.614]     243216266
Stop value for last time bin in MET[243217766.614]     243218266
Width of linearly uniform time bins in seconds[4.096]     10


gtbin: WARNING: No spacecraft file: EXPOSURE keyword will be set equal to ontime.


**Notes**:
* We used **gtselect** to create `GRB081916C_lc_events.fits`, a photon list from a small region but using a long time range.


* **gtbin** binned these photons and output the result into `GRB081916C_light_curve.fits`.

  There are a number of options for choosing the time bins. Here we have chosen linear bins with equal time widths of 10 seconds.

## Examine the Data

We now have two FITS files with binned data: one with a lightcurve (`GRB081916C_light_curve.fits`), and a second with a counts map (`GRB081916C_counts_map.fits`).

**Note**: Currently we do not have any Fermi specific graphics programs, but there are various tools available to plot FITS data files, such as [_fv_](http://heasarc.nasa.gov/ftools/fv/) and [_ds9_](http://hea-www.harvard.edu/RD/ds9/).

To look at the counts map, we will use *fv*.

1. First, start up *fv* in your terminal using:
       prompt> fv &
    (Or see the code cell below)
       
       
2. Then open `GRB081916C_counts_map.fits`. Click on 'open file' to get the `File Dialog` GUI. Choose `GRB081916C_counts_map.fits`. A new GUI will open up with a table with two rows.


3. FITS files consist of a series of extensions with data. Since the counts map is an image, it is stored in the primary extension (for historical reasons only images can be stored in the primary extension). Clicking on the 'Image' button in the first row results in:

In [9]:
!fv ./data/GRB081916C_counts_map.fits

zsh:1: command not found: fv


In [10]:
from IPython.display import HTML

In [11]:
HTML("<img src='https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/images/explore_data_GRB/GRB081916C_counts_map_fv.png'>")

One readily observes that there are many pixels containing a few counts each near the burst location, and few pixels containing any counts away from the burst. It is also evident that the burst location is not centered in the counts map. This means the preliminary location sent out in the GCN was not quite the right location. This is not uncommon for automated transient localizations.

Using *ds9* instead of *fv* allows us to find the location of the brightest pixel, and a better position for the burst.

In [12]:
HTML("<img src='https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/images/explore_data_GRB/GRB081916C_counts_map_ds9.png'>")

By simply mousing over that pixel, the RA and Dec of the likely burst location get displayed in the FK5 boxes (here, the new position is 119.5, -56.5). This is a quick method of localizing a burst, though not as accurate as fitting the data (described in the [Likelihood Tutorial](https://fermi.gsfc.nasa.gov/ssc/data/p6v11/analysis/scitools/likelihood_tutorial.html)).

> **Note**: To change the *ds9* display for the mouseover from sexagesimal (the default) to degrees, go to the WCS menu and select "Degrees" in the bottom portion of the menu.

**To look at the lightcurve:**

1. Open `GRB081916C_light_curve.fits` in *fv*.

    The lightcurve is in the RATE extension. Choose 'All' to view the content of that extension. The extension has 4 columns: TIME, TIMEDEL, COUNTS and ERROR.
    
    The time bins were created to have a width of 10 seconds, and therefore TIMEDEL is always 10.
    
2. Now plot the COUNTS as a function of TIME. Select 'Plot' for the RATE extension. Click on 'Time' then on 'x'. Click on 'Counts' then on 'y'. Finally click on 'Plot'. The result is:

In [13]:
!fv ./data/GrB081916C_light_curve.fits

zsh:1: command not found: fv


In [14]:
HTML("<img src='https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/images/explore_data_GRB/GRB081916C_light_curve_fv.png'>")

>**Note**: Except for the period of the burst, almost all bins have 0-5 counts. This demonstrates that there is very little background for LAT observations of gamma-ray bursts.